In [1]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import ipeadatapy as ipea
from bcb import sgs
# Dados do saldo do CAGED (Ipeadata)
dados_ipeadata = (
  ipea.timeseries("CAGED12_SALDON12")
  .rename_axis("data", axis = "index")
  .rename(columns = {"VALUE (Pessoa)": "caged"})
  .filter(items = ["caged"], axis = "columns")
  .query("data <= '2020-06-01'")
)
# Dados do IDP/BP - acum. 12m - US$ (milhões) (SGS/BCB)
dados_sgs = (
  sgs.get(
    codes = {"idp": 24422}, 
    start = "2019-11-01", 
    end = "2020-03-01")
  .rename_axis("data", axis = "index")
)
# Cruzamento de dados
tabela = dados_ipeadata.join(other = dados_sgs, how = "outer")

In [3]:
tabela

,caged,idp
data,,
2019-11-01,NaN,74643.3
2019-12-01,NaN,69174.4
2020-01-01,92678.0,69231.7
2020-02-01,198368.0,65058.0
2020-03-01,-232316.0,69231.5
2020-04-01,-902317.0,NaN
2020-05-01,-352790.0,NaN
2020-06-01,-23111.0,NaN


In [5]:
# Testa quais observações são NaN
tabela.isna()

,caged,idp
data,,
2019-11-01,True,False
2019-12-01,True,False
2020-01-01,False,False
2020-02-01,False,False
2020-03-01,False,False
2020-04-01,False,True
2020-05-01,False,True
2020-06-01,False,True


In [10]:
# Filtrando linhas com NaN (uma única coluna)
tabela.query("caged.isna()")

,caged,idp
data,,
2019-11-01,NaN,74643.3
2019-12-01,NaN,69174.4


In [11]:
# Filtrando linhas com NaN (+ de 1 coluna)
tabela.query("caged.isna() or idp.isna()")

,caged,idp
data,,
2019-11-01,NaN,74643.3
2019-12-01,NaN,69174.4
2020-04-01,-902317.0,NaN
2020-05-01,-352790.0,NaN
2020-06-01,-23111.0,NaN


In [13]:
tabela.apply(lambda x: np.sum(x.isna()))

caged    2
idp      3
dtype: int64

In [15]:
# Removendo linhas com NaN (de uma única coluna)
tabela.dropna(subset = ["caged"])

,caged,idp
data,,
2020-01-01,92678.0,69231.7
2020-02-01,198368.0,65058.0
2020-03-01,-232316.0,69231.5
2020-04-01,-902317.0,NaN
2020-05-01,-352790.0,NaN
2020-06-01,-23111.0,NaN


In [17]:
tabela.dropna()

,caged,idp
data,,
2020-01-01,92678.0,69231.7
2020-02-01,198368.0,65058.0
2020-03-01,-232316.0,69231.5


In [25]:
( # Substituindo valores NaN
  tabela
  # substituir NaN pela média histórica
  .fillna(value = {"caged": tabela.caged.mean()})
  # substituir NaN pelo último valor observado
  .ffill()
)

,caged,idp
data,,
2019-11-01,-203248.0,74643.3
2019-12-01,-203248.0,69174.4
2020-01-01,92678.0,69231.7
2020-02-01,198368.0,65058.0
2020-03-01,-232316.0,69231.5
2020-04-01,-902317.0,69231.5
2020-05-01,-352790.0,69231.5
2020-06-01,-23111.0,69231.5


In [30]:
#Outra possibilidade é o uso da função combine_first() 
# do pandas que possibilita substituir NaN de uma coluna 
# com base em valores ao lado (de outra coluna):
tabela_test=tabela.dropna()
tabela_test.assign(idp_2=tabela_test['caged']**2)




,caged,idp,idp_2
data,,,
2020-01-01,92678.0,69231.7,8.589212e+09
2020-02-01,198368.0,65058.0,3.934986e+10
2020-03-01,-232316.0,69231.5,5.397072e+10
